# competition_report.ipynb
pulls data from craigslist and generate reports.

## create and activate environment

### run this in terminal
```bat
conda create -n compr 
conda activate compr
pip install bs4
```

### or just activate environment
```bat
conda activate compr
```

### documentation for python-craigslist
https://pypi.org/project/python-craigslist/

## get data

In [9]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# pd.set_option('display.min_rows', 100)

# from craigslist import CraigslistServices

from craigslist import data as cl_data
import craigslist as cl
%matplotlib inline

from pprint import pprint


In [7]:
pprint(cl_data.get_areas())

{'abbotsford': {'abbreviation': 'abb',
                'area_id': 471,
                'country': 'CA',
                'description': 'fraser valley, BC',
                'hostname': 'abbotsford',
                'latitude': 49.051399,
                'longitude': -122.278999,
                'region': 'BC',
                'short_description': 'fraser valley',
                'timezone': 'America/Vancouver',
                'tld': 'ca'},
 'aberdeen': {'abbreviation': 'abz',
              'area_id': 318,
              'country': 'GB',
              'description': 'aberdeen',
              'hostname': 'aberdeen',
              'latitude': 57.150002,
              'longitude': -2.1,
              'region': '',
              'short_description': 'aberdeen',
              'timezone': 'Europe/London',
              'tld': 'co.uk'},
 'abilene': {'abbreviation': 'abi',
             'area_id': 364,
             'country': 'US',
             'description': 'abilene, TX',
             'hostname

In [8]:
pprint(cl_data.get_categories())

{'aap': {'abbreviation': 'aap',
         'category_id': None,
         'description': 'all apartments',
         'type': 'H'},
 'abo': {'abbreviation': 'abo',
         'category_id': 86,
         'description': 'apts by owner',
         'type': 'H'},
 'acc': {'abbreviation': 'acc',
         'category_id': 23,
         'description': 'accounting/finance',
         'type': 'J'},
 'act': {'abbreviation': 'act',
         'category_id': 35,
         'description': 'activity partners',
         'type': 'C'},
 'aos': {'abbreviation': 'aos',
         'category_id': 106,
         'description': 'automotive services',
         'type': 'B'},
 'apa': {'abbreviation': 'apa',
         'category_id': 1,
         'description': 'apts/housing for rent',
         'type': 'H'},
 'app': {'abbreviation': 'app',
         'category_id': 149,
         'description': 'appliances - by owner',
         'type': 'S'},
 'ard': {'abbreviation': 'ard',
         'category_id': 170,
         'description': 'arts & craf

In [10]:
for post in cl.search('washingtondc', 'apa', postal=20071, search_distance=1):
    print(post)

KeyError: 'Ask'

In [13]:
post = cl.get('https://charlotte.craigslist.org/search/charlotte-nc/roo?lat=35.21870&lon=-80.80990&search_distance=17')

AttributeError: 'NoneType' object has no attribute 'groups'

In [2]:
from craigslist import CraigslistHousing
cl_h = CraigslistHousing(site='sfbay', area='sfc', category='roo')

print(cl_h.get_results_approx_count())

for result in cl_h.get_results():
    print(result)

None


AttributeError: 'NoneType' object has no attribute 'find_all'

In [15]:
CraigslistServices.show_categories()

NameError: name 'CraigslistServices' is not defined

In [ ]:
def clr_to_df(clr):
    t = []
    for result in cl_h.get_results(sort_by='newest', geotagged=True):
        t.append(result)
    return pd.DataFrame(t)

In [ ]:
from craigslist import CraigslistHousing
cl_h = CraigslistHousing(site='charlotte',category='roo',
                        filters={
                            'max_price': 5000, 
                            'min_price':500, 
                            'private_room': True,
                            'private_bath': True,
                            # 'dogs_ok':1,
                            # 'cats_ok':1,
                            })
df = clr_to_df(cl_h)

cl_h = CraigslistHousing(site='charlotte', area='charlotte', category='roo',
                        filters={
                            'max_price': 5000, 
                            'min_price':500, 
                            'private_room': True,
                            'private_bath': True,
                            'dogs_ok':1,
                            # 'cats_ok':1,
                            })
df_d = clr_to_df(cl_h)



In [ ]:
def dollar_to_int(x):
    try:
        x = x.replace('$','')
        x = x.replace(',','')
        return int(x)
    except:
        return x

In [ ]:

# display(len(df))
# display(df.head(10))
# df.to_csv('data.csv')

df['price'] = df['price'].apply(dollar_to_int)
df_d['price'] = df_d['price'].apply(dollar_to_int)

print('***no dog***')
print(df['price'].describe())
print('')
print('***with dog***')
print(df_d['price'].describe())


In [ ]:


def pivotize(_df):
    dp = pd.pivot_table(
        _df, 
        index=['price'],
        aggfunc=len
        )
    dp = dp.reset_index()
    dp = dp[['price','name']]
    # dp['price'] = dp['price'].apply(dollar_to_int)
    dp = dp.rename(columns={'name':'count'})
    dp = dp.sort_values(by=['price'],ascending=False)
    # display(df_pivot)
    return dp

df_pivot = pivotize(df)
dfd_pivot = pivotize(df_d)



In [ ]:
def break_down_buckets(idf,column,buckets,message='',nan_value=-1):
    """
    breaks a column down into buckets/bins
    """
    idf = idf.fillna(nan_value)

    print('',message,'\ncolumn: ',column, '\nbuckets: ', buckets)
    
    idf = pd.DataFrame(idf[column])
    idf['bucket'] = pd.cut(idf[column], bins=buckets)
    idf = idf.groupby(by='bucket').count()
    idf['percent'] = (idf[column]/idf[column].sum())*100
    idf['percent'] = idf['percent'].round(2)
    # display(idf)
    return idf

In [ ]:
buckets = list(range(200,2000,100))
# print(buckets)
df_bucket = break_down_buckets(df_pivot,'price',buckets)
df_bucket['price'].plot.bar()
# print('\n')
# print('\n' + '-'*25 + '\n')
# df_bucket['percent'].plot.bar()
# print('\n' + '-'*25 + '\n')

In [ ]:
buckets = list(range(200,2000,100))
# print(buckets)
df_bucket = break_down_buckets(dfd_pivot,'price',buckets)
df_bucket['price'].plot.bar()
# df_bucket['percent'].plot.bar()